In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import pickle
import os

In [2]:
flight_data = (
    pd.read_csv('final_dataset_cleaned_v1.csv')
)

In [3]:
flight_data.shape

(12658696, 29)

In [4]:
flight_data.head(n = 3)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,TAIL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,...,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,coco,target
0,2020,1,1,1,3,2020-01-01,N951WN,ONT,"Ontario, CA",SFO,...,2020-01-01 18:00:00,12.2,-7.3,25.0,0.0,330.0,18.4,1017.3,3.0,1
1,2020,1,1,1,3,2020-01-01,N467WN,ONT,"Ontario, CA",SFO,...,2020-01-01 11:00:00,3.9,-3.4,59.0,0.0,210.0,5.4,1017.6,2.0,0
2,2020,1,1,1,3,2020-01-01,N7885A,ONT,"Ontario, CA",SJC,...,2020-01-01 20:00:00,12.8,-7.3,24.0,0.0,290.0,11.2,1015.8,2.0,0


In [5]:
flight_data['aircraft_type_new'] = [i.split('-')[0] for i in flight_data['Aircraft type']]

In [6]:
# list of numerical and categorical features to select 
l_numeric_cols = [ 'DEP_HOUR', 'temp', 'dwpt', 'rhum', 'prcp', 'wspd', 'pres', 'coco']
l_categorical_cols = ['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'aircraft_type_new']
target = "target"

In [7]:
# converting the categorical columns to categorical
for col in l_categorical_cols:
    flight_data[col] = flight_data[col].astype('category')

In [8]:
flight_data = flight_data[l_numeric_cols + l_categorical_cols + [target] + ['YEAR']]

In [9]:
# getting dummu values for categorical columns
QUARTER_dummies = pd.get_dummies(flight_data['QUARTER'], prefix='QUARTER', drop_first=True)
MONTH_dummies = pd.get_dummies(flight_data['MONTH'], prefix='MONTH', drop_first=True)
DAY_OF_MONTH_dummies = pd.get_dummies(flight_data['DAY_OF_MONTH'], prefix='DAY_OF_MONTH', drop_first=True)
DAY_OF_WEEK_dummies = pd.get_dummies(flight_data['DAY_OF_WEEK'], prefix='DAY_OF_WEEK', drop_first=True)
Aircraft_type_dummies = pd.get_dummies(flight_data['aircraft_type_new'], prefix='Aircraft_type', drop_first=True)

In [10]:
# Aircraft_type_dummies.columns = ['Aircraft_{}'.format(i) for i in range(len(Aircraft_type_dummies.columns))]

In [11]:
flight_data = flight_data.drop(l_categorical_cols, axis=1)

In [12]:
dummy_flight_data = pd.concat([flight_data, QUARTER_dummies, MONTH_dummies,
                               DAY_OF_MONTH_dummies, DAY_OF_WEEK_dummies, Aircraft_type_dummies ], axis=1)

In [13]:
del Aircraft_type_dummies,QUARTER_dummies,MONTH_dummies,DAY_OF_MONTH_dummies,DAY_OF_WEEK_dummies

In [14]:
train = dummy_flight_data[dummy_flight_data['YEAR'] == 2019]
test = dummy_flight_data[dummy_flight_data['YEAR'] == 2020]

In [15]:
del flight_data, dummy_flight_data

In [16]:
del train

In [17]:
train = pd.read_csv('undersampled_training_data_2019.csv')

In [ ]:
# train = train.drop(['Unnamed: 0'], axis=1)

In [18]:
train.head()

,DEP_HOUR,temp,dwpt,rhum,prcp,wspd,pres,coco,QUARTER_2,QUARTER_3,...,Aircraft_type_Piper PA,Aircraft_type_Piper PA22,Aircraft_type_Quest Kodiak 100,Aircraft_type_Robinson R66 Turbine,Aircraft_type_Rotorcraft,Aircraft_type_Saab 2000,Aircraft_type_Socata TBM,Aircraft_type_Van's RV,Aircraft_type_ Boeing 767,target
0,13.0,12.2,10.6,90.0,0.0,0.0,1017.9,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,9.0,6.7,6.1,96.0,0.0,5.4,1006.3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,17.0,26.7,13.4,44.0,0.0,27.7,1013.4,3.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,12.0,9.4,9.4,100.0,0.0,20.5,1021.6,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,13.0,11.1,7.2,77.0,0.0,20.5,1024.8,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [19]:
logistic_regression = LogisticRegression(multi_class="ovr")
logistic_regression.fit(
    train.drop(["target"], axis=1).values,
     train[target]
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

LogisticRegression(multi_class='ovr')

In [20]:
path = "./models"
filename = "2019train_LR_ovr_undersample.mdl"

In [21]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(logistic_regression, f, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
# predicting the test results 
y_pred = logistic_regression.predict(test.drop(["YEAR","target"], axis=1).values)

In [23]:
print(confusion_matrix(test[target],y_pred))

[[2497686  623596  458391  724710]
 [ 115690   56153   45405   59810]
 [  30662   13402   17452   23950]
 [  19117    6582   12083   16544]]


In [24]:
print(classification_report(test[target],y_pred))

              precision    recall  f1-score   support

           0       0.94      0.58      0.72   4304383
           1       0.08      0.20      0.11    277058
           2       0.03      0.20      0.06     85466
           3       0.02      0.30      0.04     54326

    accuracy                           0.55   4721233
   macro avg       0.27      0.32      0.23   4721233
weighted avg       0.86      0.55      0.66   4721233



In [25]:
print("Testing  set score for model: %f" % logistic_regression.score(test.drop(["YEAR","target"], axis=1).values  , test[target] ))

Testing  set score for model: 0.548127


In [26]:
###### logistic regression multinomial ####
logistic_regression_mul = LogisticRegression(multi_class="multinomial")
logistic_regression_mul.fit(
    train.drop(["target"], axis=1).values,
     train[target]
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(multi_class='multinomial')

In [27]:
filename = "2019train_LR_mul_undersample.mdl"

In [28]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(logistic_regression_mul, f, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
# predicting the test results 
y_pred = logistic_regression_mul.predict(test.drop(["YEAR","target"], axis=1).values)

In [30]:
print(confusion_matrix(test[target],y_pred))

[[2527095  491910  680270  605108]
 [ 119866   46929   68237   42026]
 [  32341   13785   25384   13956]
 [  20441    8244   17285    8356]]


In [31]:
print(classification_report(test[target],y_pred))

              precision    recall  f1-score   support

           0       0.94      0.59      0.72   4304383
           1       0.08      0.17      0.11    277058
           2       0.03      0.30      0.06     85466
           3       0.01      0.15      0.02     54326

    accuracy                           0.55   4721233
   macro avg       0.27      0.30      0.23   4721233
weighted avg       0.86      0.55      0.67   4721233



In [32]:
print("Testing  set score for model: %f" % logistic_regression_mul.score(test.drop(["YEAR","target"], axis=1).values  , test[target] ))

Testing  set score for model: 0.552348
